In [18]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer

import unicodedata as ud
from sklearn.metrics import precision_recall_fscore_support as prfs
import csv
from itertools import izip
import HTMLParser
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn import linear_model
from sklearn.ensemble import AdaBoostClassifier

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer as porterStemmer

In [19]:
data = pd.read_excel("C:\\Users\\Khushbu\\Desktop\\uic\\main\\Data Mining\\Project 2\\romneyTweets_Filtered.xlsx");
data.head(5)

,date,time,Anootated tweet,Class
0,NaN,NaN,"1: positive, -1: negative, 0: neutral, 2: mixed",NaN
1,2012-10-16 00:00:00,09:38:08-05:00,Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...,-1.0
2,2012-10-16 00:00:00,10:14:18-05:00,.@WardBrenda @shortwave8669 @allanbourdius you...,-1.0
3,2012-10-16 00:00:00,09:27:16-05:00,<e>Mitt Romney</e> still doesn't <a>believe</a...,-1.0
4,2012-10-16 00:00:00,10:11:43-05:00,<e>Romney</e>'s <a>tax plan</a> deserves a 2nd...,-1.0


In [20]:
annotatedTweet = data['Anootated tweet']
actualClass = data['Class']
lengthObama = len(data);

annotatedTweet = annotatedTweet.drop(0)
actualClass = actualClass.drop(0)


annotatedTweetList = annotatedTweet.tolist();
actualClassList = actualClass.tolist();

# For test data

data = pd.read_excel("C:\\Users\\swapnil sagar\\Documents\\UIC\\Data mining\\project 2\\testData.xlsx");
annotatedTweet = data['Anootated tweet']
annotatedTweet = annotatedTweet.drop(0)
annotatedTweetList = annotatedTweet.tolist();

In [21]:
count = 0;

finalFilteredTweet = []

def removeStopWords(tweet):
    filtered_tweet = [];
    
    stop_words = set(stopwords.words('english'));
    word_tokens = word_tokenize(tweet);
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_tweet.append(w);
    
    eachTweet = " ".join(filtered_tweet)  
    return eachTweet
    
    
def preProcessTweets(annotatedTweet):
        
        #remove links
        processedTweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', annotatedTweet);
        #remove references with @<word>
        processedTweet = re.sub(r'(\s)@\w+', r'', processedTweet)
        processedTweet = re.sub(r'@\w+', r'', processedTweet)
        
        # remove HTML tags from tweet
        processedTweet = re.sub('<[^<]+?>', '', processedTweet)
        
        #remove exclamation marks - hashtags should be removed?
        processedTweet = re.sub(r'[<>!#@$:,%\?-]+', r' ', processedTweet)
        processedTweet = re.sub(r'[.]+', r'', processedTweet)
        
       # processedTweet = re.sub(r'(\s)#\w+', r'', processedTweet)
       # processedTweet = re.sub(r'#\w+', r'', processedTweet)
        
        # remove extra white spaces
        processedTweet = re.sub(r'\s+', r' ', processedTweet)
        
        #remove "" 
        processedTweet = processedTweet.replace('"', '');
        
        processedTweet = ''.join([i if ord(i) < 128 else ' ' for i in processedTweet])

        #stemming
        stemmer = porterStemmer()
        stemmedTweet = [stemmer.stem(word) for word in processedTweet.split(" ")]
        stemmedTweet = " ".join(stemmedTweet)
        processedTweet = str(stemmedTweet);
        
        processedTweet = processedTweet.replace("'", "");

        #remove numbers from data
        #join
        #processedTweet = " ".join(re.findall('[A-Z][^A-Z]*', processedTweet));

        #processedTweet = removeStopWords(processedTweet); --- Remove stop words in the end
        
        return processedTweet;

    
for every_tweet in annotatedTweetList:
    
    count = count +1     
    tweet = preProcessTweets(every_tweet).encode('ascii', 'ignore').strip();
    #print type(tweet)

    finalFilteredTweet.append(tweet);        

In [22]:
len(finalFilteredTweet)

5648

In [23]:
finalFilteredTweet[:5]

['insidi mitt romney bain help philip morri get US high schooler hook On cigarett via',
 'you mean like romney cheat in primari',
 'mitt romney still doesnt believ that we have a black presid',
 'romney tax plan deserv a 2nd look becaus he ha a secret one that differ than the one he been lie about',
 'hope romney debat prep w/ the same peopl as last time']

In [7]:
trainData, testTrainData , y_train, y_test = train_test_split(finalFilteredTweet, actualClassList, test_size = 0.15, random_state = 40)

In [8]:
len(trainData)
len(testTrainData)

848

In [24]:
vectorizer = TfidfVectorizer(min_df=0.00125,
                             max_df = 0.70,
                             sublinear_tf=True,
                             use_idf=True,
                            stop_words=u'english',
                            analyzer='word',
                            ngram_range=(1,6),lowercase=True)

In [25]:
#train_vectors = vectorizer.fit_transform(trainData);
#test_vectors = vectorizer.transform(testTrainData);
total_vectors = vectorizer.fit_transform(finalFilteredTweet);

#knn = KNeighborsClassifier()
classifier_sgd = linear_model.SGDClassifier()

In [26]:
classifier_sgd.fit(total_vectors, actualClassList)
#knn.fit(train_vectors, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [12]:
predicted_labels = classifier_sgd.predict(test_vectors)
#predicted_labels = knn.predict(test_vectors)
predicted_labelsList = predicted_labels.tolist();
classifier_sgd.score(test_vectors, y_test)

0.56603773584905659

In [14]:
target_classifiers = ['-1', '0', '1']
print(classification_report(y_test, predicted_labelsList, target_names=target_classifiers))

             precision    recall  f1-score   support

         -1       0.62      0.77      0.69       425
          0       0.42      0.33      0.37       252
          1       0.57      0.39      0.46       171

avg / total       0.55      0.57      0.55       848



# Get Predicted list for test Data

In [14]:
submission = pd.DataFrame(columns=['id','Class'])
submission

,id,Class


In [15]:
i = 0
for num in predicted_labelsList:
    submission.loc[i+1] = [i+1,predicted_labelsList[i]]

    i=i+1

In [16]:
np.savetxt(r'C:\Users\swapnil sagar\Documents\UIC\Data mining\project 2\output_Romney.txt', submission.values, fmt='%d')
submission.head(5)

,id,Class
1,1.0,-1.0
2,2.0,-1.0
3,3.0,-1.0
4,4.0,0.0
5,5.0,-1.0


In [15]:
cvScores = cross_val_score(classifier_sgd, total_vectors, actualClassList, cv=10,scoring='f1_weighted')
print (cvScores)

[ 0.56838101  0.50632915  0.52121745  0.53164086  0.53960149  0.56352106
  0.55873243  0.49270258  0.54302728  0.58808493]


In [16]:
print ("Mean: ", cvScores.mean())
print ("Minimum: ", cvScores.min())
print ("Maximum: ", cvScores.max())

('Mean: ', 0.54132382496854459)
('Minimum: ', 0.49270258339693024)
('Maximum: ', 0.58808492664499701)


In [17]:
cvPredict = cross_val_predict(classifier_sgd, total_vectors, actualClassList, cv=10)
print (cvPredict)

[-1. -1. -1. ..., -1. -1. -1.]


In [18]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.62      0.76      0.69      2893
        0.0       0.41      0.32      0.36      1680
        1.0       0.51      0.40      0.45      1075

avg / total       0.54      0.56      0.54      5648



ADA BOOST WITH SVM

In [19]:
bdt = AdaBoostClassifier(classifier_sgd,
                         algorithm="SAMME",
                         n_estimators=200)

In [20]:
cvPredict = cross_val_predict(bdt, total_vectors, actualClassList, cv=10)
print (cvPredict)

[-1. -1. -1. ..., -1.  0. -1.]


In [21]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.63      0.63      0.63      2893
        0.0       0.37      0.37      0.37      1680
        1.0       0.41      0.40      0.40      1075

avg / total       0.51      0.51      0.51      5648



FOR TEST DATA

In [27]:
dataNew = pd.read_excel("C:\\Users\\Khushbu\\Desktop\\uic\\main\\Data Mining\\Project 2\\sample-testdata.xlsx",sheetname = "Romney");
print dataNew

                                     Anootated tweet
1  Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...
2  Senior <e>Romney</e> Advisor Claims <e>Obama</...
3  .@WardBrenda @shortwave8669 @allanbourdius you...
4  <e>Mitt Romney</e> still doesn't <a>believe</a...


In [28]:
annotatedTweetTest = dataNew['Anootated tweet']
#annotatedTweetTest = annotatedTweet.drop(0)
annotatedTweetListTest = annotatedTweetTest.tolist();
print annotatedTweetListTest

[u"Insidious!<e>Mitt Romney</e>'s Bain Helped Philip Morris Get U.S. High Schoolers <a>Hooked On Cigarettes</a> http://t.co/nMKuFcUq via @HuffPostPol", u'Senior <e>Romney</e> Advisor Claims <e>Obama</e> Administration Is Deliberately <a>Misleading Public On Libya</a>: http://t.co/cpJjGsCF tp #US', u'.@WardBrenda @shortwave8669 @allanbourdius you mean like <e>romney </e><a>cheated in primary</a>?', u"<e>Mitt Romney</e> still doesn't <a>believe</a> that we <a>have a black president</a>."]


In [29]:
count = 0;

finalFilteredTweet = []

def removeStopWords(tweet):
    filtered_tweet = [];
    
    stop_words = set(stopwords.words('english'));
    word_tokens = word_tokenize(tweet);
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_tweet.append(w);
    
    eachTweet = " ".join(filtered_tweet)  
    return eachTweet
    
    
def preProcessTweets(annotatedTweet):
        
        #remove links
        processedTweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', annotatedTweet);
        #remove references with @<word>
        processedTweet = re.sub(r'(\s)@\w+', r'', processedTweet)
        processedTweet = re.sub(r'@\w+', r'', processedTweet)
        
        # remove HTML tags from tweet
        processedTweet = re.sub('<[^<]+?>', '', processedTweet)
        
        #remove exclamation marks - hashtags should be removed?
        processedTweet = re.sub(r'[<>!#@$:,%\?-]+', r' ', processedTweet)
        processedTweet = re.sub(r'[.]+', r'', processedTweet)
        
       # processedTweet = re.sub(r'(\s)#\w+', r'', processedTweet)
       # processedTweet = re.sub(r'#\w+', r'', processedTweet)
        
        # remove extra white spaces
        processedTweet = re.sub(r'\s+', r' ', processedTweet)
        
        #remove "" 
        processedTweet = processedTweet.replace('"', '');
        
        processedTweet = ''.join([i if ord(i) < 128 else ' ' for i in processedTweet])

        #stemming
        stemmer = porterStemmer()
        stemmedTweet = [stemmer.stem(word) for word in processedTweet.split(" ")]
        stemmedTweet = " ".join(stemmedTweet)
        processedTweet = str(stemmedTweet);
        
        processedTweet = processedTweet.replace("'", "");

        #remove numbers from data
        #join
        #processedTweet = " ".join(re.findall('[A-Z][^A-Z]*', processedTweet));

        #processedTweet = removeStopWords(processedTweet); --- Remove stop words in the end
        
        return processedTweet;

    
for every_tweet in annotatedTweetListTest:
    
    count = count +1     
    tweet = preProcessTweets(every_tweet).encode('ascii', 'ignore').strip();
    #print type(tweet)

    finalFilteredTweet.append(tweet);        

APPLY VECTORIZER ON TEST DATA

In [30]:
test_vectors_new = vectorizer.transform(finalFilteredTweet);
test_vectors_new

<4x1600 sparse matrix of type '<type 'numpy.float64'>'
	with 38 stored elements in Compressed Sparse Row format>

In [33]:
predicted_labels = classifier_sgd.predict(test_vectors_new)
#predicted_labels = knn.predict(test_vectors)
predicted_labelsList = predicted_labels.tolist();
predicted_labelsList

[-1.0, -1.0, -1.0, -1.0]

GET DATA ON THE OUTPUT FILE

In [34]:
submission = pd.DataFrame(columns=['id','extra_column_for_tab','Class'])
#submission.loc[0] = [1, 2, 3]
submission

,id,extra_column_for_tab,Class


In [36]:
i = 0
for num in predicted_labelsList:
    submission.loc[i+1] = [i+1,';;',predicted_labelsList[i]]

    i=i+1

In [37]:
submission.head(5)
np.savetxt(r'C:\Users\Khushbu\Desktop\uic\main\Data Mining\Project 2\output_romney.txt', submission.values, fmt='%d%s%d')